Pros:
    Very accurate and robust
    Does not suffer from overfitting
    Can do feature selection
Cons:
    Computational complexity
    Difficult to interpret?


In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



In [3]:
df = pd.read_csv('./Data Exploration/wdbc.csv')
df = df.drop([["Area", "AreaSE", "AreaWorst", "Perimeter", "PerimeterSE", "PerimeterWorst"]], axis = 1)
encoder = LabelEncoder().fit(df["B/M"])
df['B/M'] = encoder.transform(df["B/M"])
not_data = ["ID", "B/M"]
label = df["B/M"]
feature = df.drop(not_data, axis = 1)
data = pd.concat([label, feature], axis=1)

KeyError: "[('Area', 'AreaSE', 'AreaWorst', 'Perimeter', 'PerimeterSE', 'PerimeterWorst')] not found in axis"

No need to standardise as not sensitive to magnitude

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.25, stratify=label)

In [ ]:
# ROC ROC CURE, AUC CURVE

rfc = RandomForestClassifier(random_state=0, n_estimators=10)
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)

print(accuracy_score(y_test, y_pred))
feature_scores = pd.Series(rfc.feature_importances_, index=x_train.columns).sort_values(ascending=False)
print(feature_scores)

In [ ]:


cm = confusion_matrix(y_test, y_pred)

print('Confusion matrix\n\n', cm)

print(classification_report(y_test, y_pred))

In [ ]:
feature_scores = feature_scores.index.tolist()
important_features = feature_scores[:-10]
print(important_features)
feature = df[important_features]
print(feature)
x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.25, stratify=label)



In [ ]:
rfc = RandomForestClassifier(random_state=0, n_estimators=10)
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)

print(accuracy_score(y_test, y_pred))
feature_scores = pd.Series(rfc.feature_importances_, index=x_train.columns).sort_values(ascending=False)

